 计算机视觉（24春）作业1-4 (25分)
---

## 题目: DLT算法的使用
- **给定理想二维匹配点对集合，请使用DLT算法估计其所对应的单应性矩阵H。**
- 二维匹配点数据文件名为：2D_correspondences4DLT_H.txt
    - 存储格式为：每行4个数，分别为 (x1, y1, x2, y2)，表示两个平面间的匹配点坐标。
- 请在下面的代码框中编写DLT算法的实现，并输出估计得到的单应性矩阵H。

---
### 环境配置：以anaconda为例

```shell
    # anaconda的安装请根据自己的操作系统下载对应的安装包，官网链接：https://www.anaconda.com/products/distribution
    # 具体安装过程可以参考知乎，b站，CSDN等网站上的安装教程
    # 假设你已经成功安装了anaconda，打开terminal（linux/mac）or Anaconda Prompt（windows）
    cd <你的文件夹路径>
    conda create -n cv python=3.8 # 创建python虚拟环境，名字为cv，python版本为3.8
    conda activate cv # 激活虚拟环境
    conda install jupyter notebook # 安装jupyter notebook
    pip install --upgrade pip # 升级pip
    pip install opencv-python # 安装opencv
    pip install numpy # 安装numpy
    jupyter notebook # 运行jupyter notebook
    # 上一条指令将在默认的网页浏览器中开启一个新的工作空间，你可以在其中打开该.ipynb文件
```

---

## 数据读取与展示部分
- 请在下面的代码框中实现对数据的读取与展示。
- 评分细则：
    1.数据读取与展示的正确性。（5分）

In [1]:
# Code here
# hint: use np.loadtxt

# Define the file path
file_path = r'C:\Users\ASUS\Desktop\计算机视觉\HW1\2D_correspondences4DLT_H.txt'

# Read the file and parse the points
points1 = []
points2 = []

with open(file_path, 'r') as file:
    for line in file:
        x1, y1, x2, y2 = map(float, line.split())
        points1.append([x1, y1])
        points2.append([x2, y2])

# Display the data
print('Points in the first image:')
print(points1)
print('Points in the second image:')
print(points2)


Points in the first image:
[[54.88135039273247, 71.51893663724195], [60.276337607164386, 54.48831829968969], [42.36547993389047, 64.58941130666561], [43.75872112626925, 89.17730007820798], [96.36627605010293, 38.34415188257777], [79.17250380826646, 52.88949197529045], [56.80445610939323, 92.5596638292661], [7.103605819788694, 8.712929970154072], [2.021839744032572, 83.2619845547938], [77.81567509498505, 87.00121482468191]]
Points in the second image:
[[0.6866735123515855, 3.720218074625621], [0.7977649932521848, 3.6079297186517345], [0.644282469091861, 3.7615225341500578], [0.5734934788866926, 3.834168934491985], [1.0799308264121767, 3.3259452850839932], [0.8988403820408728, 3.507555484122706], [0.6265552397623269, 3.7816240045336915], [0.7141407606494478, 3.6444722597213075], [0.31403196429432656, 4.091628106753371], [0.7310159589293976, 3.677391087190065]]


---

## 数据处理部分
- 请在下面的代码框中实现数据的预处理，主要为齐次坐标的构造，并展示你的处理结果。
- 评分细则：
    1. 齐次坐标构造是否正确（5分）

In [2]:
# Code here
# create homogeneous coordinates
for i in range(len(points1)):
    points1[i].append(1)
    points2[i].append(1)

# Display the data
print('Points in the first image (homogeneous coordinates):')
print(points1)
print('Points in the second image (homogeneous coordinates):')
print(points2)


Points in the first image (homogeneous coordinates):
[[54.88135039273247, 71.51893663724195, 1], [60.276337607164386, 54.48831829968969, 1], [42.36547993389047, 64.58941130666561, 1], [43.75872112626925, 89.17730007820798, 1], [96.36627605010293, 38.34415188257777, 1], [79.17250380826646, 52.88949197529045, 1], [56.80445610939323, 92.5596638292661, 1], [7.103605819788694, 8.712929970154072, 1], [2.021839744032572, 83.2619845547938, 1], [77.81567509498505, 87.00121482468191, 1]]
Points in the second image (homogeneous coordinates):
[[0.6866735123515855, 3.720218074625621, 1], [0.7977649932521848, 3.6079297186517345, 1], [0.644282469091861, 3.7615225341500578, 1], [0.5734934788866926, 3.834168934491985, 1], [1.0799308264121767, 3.3259452850839932, 1], [0.8988403820408728, 3.507555484122706, 1], [0.6265552397623269, 3.7816240045336915, 1], [0.7141407606494478, 3.6444722597213075, 1], [0.31403196429432656, 4.091628106753371, 1], [0.7310159589293976, 3.677391087190065, 1]]


--- 

## DLT算法实现部分
- 请在下面的代码框中实现DLT算法，并输出估计得到的单应性矩阵H。
- 注意：
    1. 你可以直接使用cv2.findHomography()函数，如果采用此方法，请在下面的Markdown框中附上手算的过程和结果，并对比两种方法的结果。
        - 评分细则：
            - opencv函数调用是否正确（5分）
    2. 你也可以自己实现DLT算法，此时则**不需要**附上手算的过程和结果。
        - 评分细则：
            - DLT算法实现是否正确（10分）
            - 结果是否正确（5分）

In [5]:
# Code here
import numpy as np

def compute_homography(points1, points2):
    """
    Compute the homography matrix from points1 to points2 using DLT algorithm.
    
    :param points1: List of [x, y] points from the first image.
    :param points2: List of [x', y'] points from the second image.
    :return: Homography matrix H that maps points1 to points2.
    """
    if len(points1) != len(points2) or len(points1) < 4:
        raise ValueError("There must be at least 4 point correspondences.")
    
    A = []
    for (x, y, _), (xp, yp, _) in zip(points1, points2):
        A.append([-x, -y, -1, 0, 0, 0, x*xp, y*xp, xp])
        A.append([0, 0, 0, -x, -y, -1, x*yp, y*yp, yp])
    
    A = np.array(A)
    U, S, Vh = np.linalg.svd(A)
    # The homography is the last column of Vh (or the last row of V if you use V = Vh.T)
    H = Vh[-1].reshape(3, 3)
    # Normalize the homography to ensure the last element is 1
    H /= H[-1, -1]
    
    return H

# Compute the homography matrix
H = compute_homography(points1, points2)
print('Homography matrix:')
print(H)

Homography matrix:
[[1.707 0.586 1.   ]
 [2.707 8.242 2.   ]
 [1.    2.    1.   ]]


In [8]:
# # use cv2.findHomography() function
# import cv2
# H_cv, _ = cv2.findHomography(np.array(points1), np.array(points2))
# print('Homography matrix:')
# # Keep three decimal places for better display
# np.set_printoptions(precision=3)
# print(H_cv)

Homography matrix:
[[1.707 0.586 1.   ]
 [2.707 8.241 2.   ]
 [1.    2.    1.   ]]


---
*如果直接调用cv2.findHomography()函数，请在此处附上手算的过程和结果：*
- 如有需要，请熟悉Latex和Markdown的相关语法，方便书写数学公式和文字说明。
- 评分细则：
    - 算法流程正确性（4分）
    - SVD分解结果正确性（4分）
    - 两种方法结果一致性（2分）
- 以下为答题区域，可以使用多个Markdown或者Code单元格

---